In [1]:
#загружаем необходимые библиотеки
import pandas as pd
import json
import requests
import time
import traceback
import os
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from tkinter.filedialog import askdirectory

In [2]:
#функция для сохранения результата запросов FIO-DR-REGION
def results_save(js_res):
    search_name = []
    name = []
    exe_production = []
    details = []
    subject = []
    department = []
    bailiff = []
    ip_end =[]
    
    for i in js_res['response']['result']:
#обработка не найденных клиентов (вероятно в течение последних трех лет в отношении клиента не ведется исполнительного производства)
        if i['result'] == []:
                search_name.append(str((i['query']['params'])['region']) + ' ' + str((i['query']['params'])['lastname']) + ' ' + str((i['query']['params'])['firstname']) + ' ' + str((i['query']['params'])['secondname']) + ' ' + str((i['query']['params'])['birthdate']))
                name.append('empty')
                exe_production.append('empty')
                details.append('empty')
                subject.append('empty')
                department.append('empty')
                bailiff.append('empty')
                ip_end.append('empty')
        elif i['result'] == None:
                search_name.append(str((i['query']['params'])['region']) + ' ' + str((i['query']['params'])['lastname']) + ' ' + str((i['query']['params'])['firstname']) + ' ' + str((i['query']['params'])['secondname']) + ' ' + str((i['query']['params'])['birthdate']))
                name.append('empty')
                exe_production.append('empty')
                details.append('empty')
                subject.append('empty')
                department.append('empty')
                bailiff.append('empty')
                ip_end.append('empty')
#обработка некорректных входных данных (возможны проблемы с восточными или иностранными именами/либо некорректная дата рожедения)
        elif i['result'] == {'code': 0, 'message': 'The given data was invalid.', 'errors': {'number': ['Поле number имеет ошибочный формат.']}}:
                search_name.append(str((i['query']['params'])['region']) + ' ' + str((i['query']['params'])['lastname']) + ' ' + str((i['query']['params'])['firstname']) + ' ' + str((i['query']['params'])['secondname']) + ' ' + str((i['query']['params'])['birthdate']))
                name.append('Некорректные входные данные.')
                exe_production.append('Некорректные входные данные.')
                details.append('Некорректные входные данные.')
                subject.append('Некорректные входные данные.')
                department.append('Некорректные входные данные.')
                bailiff.append('Некорректные входные данные.')
                ip_end.append('Некорректные входные данные.')      
        else:
#обработка найденных клиентов (сохранение ИП, которые ведутся в отношении клиента)
            if len(i['result']) == 1:
                for j in (i['result']):
                    search_name.append(str((i['query']['params'])['region']) + ' ' + str((i['query']['params'])['lastname']) + ' ' + str((i['query']['params'])['firstname']) + ' ' + str((i['query']['params'])['secondname']) + ' ' + str((i['query']['params'])['birthdate']))
                    name.append(str((j)['name']))
                    exe_production.append((j)['exe_production'])
                    details.append((j)['details'])
                    subject.append((j)['subject'])
                    department.append((j)['department'])
                    bailiff.append((j)['bailiff'])
                    ip_end.append((j)['ip_end'])
            else:
                #корректировка списка number для компенсации дублей
                qnt = 0
                while qnt < (len(i['result'])):
                    search_name.append(str((i['query']['params'])['region']) + ' ' + str((i['query']['params'])['lastname']) + ' ' + str((i['query']['params'])['firstname']) + ' ' + str((i['query']['params'])['secondname']) + ' ' + str((i['query']['params'])['birthdate']))
                    qnt += 1
                
                for dbl in (i['result']):
                    name.append((dbl)['name'])
                    exe_production.append((dbl)['exe_production'])
                    details.append((dbl)['details'])
                    subject.append((dbl)['subject'])
                    department.append((dbl)['department'])
                    bailiff.append((dbl)['bailiff'])
                    ip_end.append((dbl)['ip_end'])
                                
#сборка результата
    query_results = pd.DataFrame({'search_name' : search_name, 'name' : name,'exe_production' : exe_production,'details' : details,'subject' : subject,'department' : department,'bailiff' : bailiff,'ip_end' : ip_end})
    return query_results

In [4]:
#выбор файла с входными данными
root = Tk()
filename = askopenfilename()
print(filename)

#выбор целевого каталога (туда мы сохраним результат)
target_path = askdirectory(title = 'Выбор каталога для выгрузки результата',initialdir = os.path.expanduser(u'C://'))
target_path = target_path.replace(r"/","\\")
root.destroy()

D:/work/FSSP/ФИО_тест/fio_realset.xlsx


In [5]:
#загружаем входные данные
ip = pd.read_excel(filename)
# total_list = ip['ip'].to_list()
print('загружено ',len(ip),' строк')

загружено  339  строк


In [6]:
#преобразование кодировки входных данных ('utf-8')
ip['fam'] = ip['fam'].apply(lambda x: bytes(x,'cp1251').decode('cp1251').encode('utf-8').decode('utf-8'))
ip['name'] = ip['name'].apply(lambda x: bytes(x,'cp1251').decode('cp1251').encode('utf-8').decode('utf-8'))
ip['sname'] = ip['sname'].fillna('')
ip['sname'] = ip['sname'].apply(lambda x: '' if x == '' else bytes(x,'cp1251').decode('cp1251').encode('utf-8').decode('utf-8'))

#приведение даты рождения в германский формат даты
ip['birth'] = pd.to_datetime(ip['birth'],errors='coerce')
ip['birth'] = ip['birth'].dt.date

In [7]:
##разметка входных данных по "батчам" - по 50 клиентов (ограничение API)
ip['batch'] = ip.index//50 + 1

In [20]:
#основной модуль

#инициализация результирующего датафрейма, инициализвция переменной, в которой хранится ключ доступа
summary = pd.DataFrame()
token = str(b'***********','utf-8')

#--------------------------------------------------------> эта часть должна быть вынесена в отдельную функцию, их будет три вида в зависимости от типа запроса, подгружаться должна будет выбранная опцию
#формирование входного JSON объекта
for i in list(ip['batch'].unique()):
    dic = {}
    rq = []
    js = ''
    print('Сессия - ',i,'\n')
    for j in ip[ip['batch'] == i].index:
        rq.append({'type':1,'params':{'firstname':ip['name'][j],'secondname':ip['sname'][j],'lastname':ip['fam'][j],'birthdate':str(ip['birth'][j]).split('-')[2] + '.' + str(ip['birth'][j]).split('-')[1] + '.' + str(ip['birth'][j]).split('-')[0],'region':int(ip['reg_number'][j])}})
    dic = {'token':token,'request':rq}
    js = json.dumps(dic)

#     print(rq)
#-------------------------------------------------------->           

#отправка группового запроса
    headers = {'accept': 'application/json','Content-Type': 'application/json'}
    try:
        response = requests.post('https://api-ip.fssp.gov.ru/api/v1.0/search/group', headers=headers, data=js)
        js_task = json.loads(response.content)
        print(js_task)
    except:
        print(sys.exc_info())
##контроль состояния предыдущей задачи / ожидание выполнения предыдущего запроса
    while js_task['exception'] == 'Дождитесь результата предыдущего группового запроса':
        time.sleep(20)
        response = requests.post('https://api-ip.fssp.gov.ru/api/v1.0/search/group', headers=headers, data=js)
        js_task = json.loads(response.content)

    headers = {'accept': 'application/json'}
    params = (('token', token),('task', (js_task['response'])['task']))

    status = requests.get('https://api-ip.fssp.gov.ru/api/v1.0/status', headers=headers, params=params)
    js_status = json.loads(status.content)
    print(js_status)
    time.sleep(3)

##проверка полноты выполнения группового запроса (надо доработать условие для неполных батчей, скорее всего проверка отрабатывает не совсем корректно или вообще эта проверка избыточна, при выгрузке результатов состояние запроса проверяется)
    while (js_status['code'] != 0) and (js_status['response']['progress']) != '50 of 50':
        time.sleep(20)
        status = requests.get('https://api-ip.fssp.gov.ru/api/v1.0/status', headers=headers, params=params)
        js_status = json.loads(status.content)
        print(js_status['code'])
        print(js_status['response']['progress'])
        
    
##получение результатов запроса
    try:
        response1 = requests.get('https://api-ip.fssp.gov.ru/api/v1.0/result', headers=headers, params=params)
        js_res = json.loads(response1.content)
    except:
        print(sys.exc_info())
    time.sleep(3)

    while (js_res['code'] != 2) and (js_res['response']['task_end'] is None):
        time.sleep(20)
        response1 = requests.get('https://api-ip.fssp.gov.ru/api/v1.0/result', headers=headers, params=params)
        js_res = json.loads(response1.content)

#проверка полноты отработки группового запроса
    status = requests.get('https://api-ip.fssp.gov.ru/api/v1.0/status', headers=headers, params=params)
    js_status = json.loads(status.content)
    print(js_status['response']['progress'],'\n')    
#этот блок обработки ответов на запрос (надо доработать)
#     if js_res['code'] == 0:
#         print('keep go on')
#         #put save function
#     if js_res['code'] == 3:
#         print('error')
#         #print errors reason
#     print('next','\n')
    
#сохранение ответа на запрос в результирующий датафрейм    
    summary = summary.append(results_save(js_res))
    
#выгрузка результатов в файл в ранее выбранный каталог
summary.to_excel(os.path.join(target_path,'summary_result_test.xlsx'))
print('Выгружено ',len(summary),' строк')

Сессия -  1 

{'status': 'success', 'code': 0, 'exception': '', 'response': {'task': '1977ee77-fac0-40bb-9067-be0124d574f9'}}
{'status': 'success', 'code': 0, 'exception': '', 'response': {'status': 2, 'progress': '0 of 50'}}
50 of 50 

Сессия -  2 

{'status': 'success', 'code': 0, 'exception': '', 'response': {'task': '66511b30-bd9b-4432-9ce9-4fd852acd593'}}
{'status': 'success', 'code': 0, 'exception': '', 'response': {'status': 1, 'progress': '0 of 50'}}
50 of 50 

Сессия -  3 

{'status': 'success', 'code': 0, 'exception': '', 'response': {'task': '91026b43-c649-435c-8b01-fddd59a2e25e'}}
{'status': 'success', 'code': 0, 'exception': '', 'response': {'status': 1, 'progress': '0 of 50'}}
50 of 50 

Сессия -  4 

{'status': 'success', 'code': 0, 'exception': '', 'response': {'task': '03c86766-b28d-47be-8afd-95f0ae39a65b'}}
{'status': 'success', 'code': 0, 'exception': '', 'response': {'status': 1, 'progress': '0 of 50'}}
50 of 50 

Сессия -  5 

{'status': 'success', 'code': 0, 'exce

In [17]:
summary = summary.drop_duplicates()